In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_json("News_Category_Dataset.json",lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [3]:
cates = df.groupby('category')
print("total categories:", cates.ngroups)
print(cates.size())

('total categories:', 31)
category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       3858
BUSINESS           4254
COLLEGE            1144
COMEDY             3971
CRIME              2893
EDUCATION          1004
ENTERTAINMENT     14257
FIFTY              1401
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
IMPACT             2602
LATINO VOICES      1129
MEDIA              2815
PARENTS            3955
POLITICS          32739
QUEER VOICES       4995
RELIGION           2556
SCIENCE            1381
SPORTS             4167
STYLE              2254
TASTE              2096
TECH               1231
THE WORLDPOST      3664
TRAVEL             2145
WEIRD NEWS         2670
WOMEN              3490
WORLD NEWS         2177
WORLDPOST          2579
dtype: int64


In [4]:
df["category"] = df["category"].map(lambda i: "WORLDPOST" if i == "THE WORLDPOST" else i)

In [5]:
df["content"]=df["headline"]+df["short_description"]
df["content"].head()

0    There Were 2 Mass Shootings In Texas Last Week...
1    Will Smith Joins Diplo And Nicky Jam For The 2...
2    Hugh Grant Marries For The First Time At Age 5...
3    Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4    Julianna Margulies Uses Donald Trump Poop Bags...
Name: content, dtype: object

In [6]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
df["content"]=df["content"].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))

In [7]:
df["content"][0]

u'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TVShe left husband. He killed children. Just another day America.'

In [8]:
x_train, x_test, y_train, y_test = train_test_split(df["content"], df["category"], test_size= 0.3, random_state=123)

In [9]:
tfidf = TfidfVectorizer(strip_accents='unicode',stop_words= 'english',ngram_range=(1,1))
tfidf.fit(x_train)
x_train_features = tfidf.transform(x_train)
x_test_features = tfidf.transform(x_test)

/usr/local/lib/python2.7/dist-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score

In [11]:
svd = TruncatedSVD(n_components=150)
x_train_features = svd.fit_transform(x_train_features)
x_test_features = svd.transform(x_test_features)

In [14]:
def accuracy(model,train_pred):
    train_accuracy = accuracy_score(y_pred=train_pred,y_true=y_train)
    test_pred=model.predict(x_test_features)
    test_accuracy = accuracy_score(y_pred=test_pred,y_true=y_test)
    return train_accuracy,test_accuracy

In [15]:
model_logistic=LogisticRegression()
model_logistic.fit(x_train_features,y_train)
pred=model_logistic.predict(x_train_features)

In [16]:
train_accuracy,test_accuracy = accuracy(model_logistic,pred)
print("train accuracy: ",train_accuracy," test accuracy: ",test_accuracy)

('train accuracy: ', 0.45349289077858546, ' test accuracy: ', 0.44958263327732884)
